In [ ]:
# -----
# IMPORT LIBRAIRIES
# -----
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import plotly.express as px
from module.pricing import DualDigitalPricer
from module.batch import DualDigitalBatchBuilder, BatchPricing

# -----
# SET DISPLAY SETTINGS
# -----
pd.set_option('display.width', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))

In [ ]:
# -----
# IMPORT DATA
# -----
df1 = pd.read_csv('data/SPY.csv', index_col=0, parse_dates=True).loc[:, ['Adj Close']]
df2 = pd.read_csv('data/IWM.csv', index_col=0, parse_dates=True).loc[:, ['Adj Close']]
df = pd.concat([df1, df2], axis=1).dropna()
df.columns = ['st1', 'st2']
df_snap = df.iloc[-252:]

In [ ]:
df_snap.pct_change().std() * np.sqrt(252)

In [ ]:
df_snap.diff().corr()

In [ ]:
# -----
# CONSTRUCT BATCH PRICING
# -----
bb_obj = DualDigitalBatchBuilder(timeseries=df_snap, roll=20)
batch_df = bb_obj.build_batch(k1=1, k2=1, iv1=0.17, iv2=0.21, q1=0, q2=0, b1=0, b2=0, direction1='up', direction2='up', rho=0.88, r=0.0, t=1, notional=100, model='numerical_integration')

In [ ]:
# -----
# PRICE
# -----
bp_obj = BatchPricing(batch=batch_df)
results = bp_obj.calculate(pricer=DualDigitalPricer, greeks_to_calculate='calculate_greeks')
results = pd.DataFrame(results).transpose()

In [ ]:
# -----
# COMPUTE PNL
# -----
pnl_df = pd.DataFrame(index=results.index)

pnl_df.loc[:, 'pv'] = results['pv'].diff()
pnl_df.loc[:, 'dst1'] = results['st1'].diff() * results['dst1'].shift()
pnl_df.loc[:, 'dst2'] = results['st2'].diff() * results['dst2'].shift()
pnl_df.loc[:, 'dst'] = pnl_df['dst1'] + pnl_df['dst2']
pnl_df.loc[:, 'dst1**2'] = results['st1'].diff().pow(2) * results['dst1**2'].shift() * 0.5
pnl_df.loc[:, 'dst2**2'] = results['st2'].diff().pow(2) * results['dst2**2'].shift() * 0.5
pnl_df.loc[:, 'dst1*dst2'] = results['st1'].diff() * results['st2'].diff() * results['dst1*dst2'].shift()
pnl_df.loc[:, 'dt'] = results['dt'].shift()
pnl_df = pnl_df.reset_index(names=['date', 'roll'])
pnl_df.loc[:, 'pe'] = pnl_df[['dst1', 'dst2', 'dst', 'dst1**2', 'dst2**2', 'dst1*dst2', 'dt']].sum(axis=1)
pnl_df = pnl_df.dropna()

In [ ]:
pnl_df.groupby('roll')[['pv', 'pe']].sum()

In [ ]:
results